# Import librairie #

In [1]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

## Charger les variables d'environnement ##

In [2]:
# Charger le .env depuis la racine du projet
from pathlib import Path
dotenv_path = r'C:\Penny\.env'
load_result = load_dotenv(dotenv_path=dotenv_path)
print(f"Chargement .env : {'[OK] Reussi' if load_result else '[ERREUR] Echec'}")

Chargement .env : ✓ Réussi


## Connexion Redshift Pennylane ##

In [3]:
conn = psycopg2.connect(
    host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
    port=5439,
    dbname='prod',
    user='u_289572',
    password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
)

print("Connexion Redshift etablie")

Connexion Redshift etablie


## Fonction helper : Connexion auto-reconnect ##

In [4]:
def get_active_connection():
    """Retourne une connexion active, la recrée si nécessaire"""
    global conn
    try:
        # Test si la connexion est active
        pd.read_sql("SELECT 1;", conn)
        return conn
    except:
        # Reconnexion si fermée
        print("⟳ Reconnexion Redshift...")
        conn = psycopg2.connect(
            host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
            port=5439,
            dbname='prod',
            user='u_289572',
            password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
        )
        print("✓ Reconnecté")
        return conn

# Utilisation : remplace "conn" par "get_active_connection()" dans tes requêtes
# Exemple :
df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())
print(f"Test réussi : {len(df)} ligne(s)")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_55928\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_55928\3220976796.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())


Test réussi : 1 ligne(s)


## Liste table Fournisseurs ##

In [5]:
# Tables suppliers Pennylane
tables_extended = [
    'suppliers'
]

accessible_tables = []

for table in tables_extended:
    try:
        query = f"SELECT * FROM pennylane.{table} LIMIT 1;"
        df = pd.read_sql(query, conn)
        accessible_tables.append({
            'table_name': table,
            'columns': len(df.columns),
            'column_list': ', '.join(df.columns.tolist()[:5]) + '...'
        })
        print(f"✓ {table} ({len(df.columns)} colonnes)")
    except:
        pass

print(f"\n\n=== RÉSUMÉ: {len(accessible_tables)} tables accessibles ===")
df_accessible = pd.DataFrame(accessible_tables)
print(df_accessible.to_string(index=False))

C:\Users\ycloa\AppData\Local\Temp\ipykernel_55928\4063426943.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ suppliers (18 colonnes)


=== RÉSUMÉ: 1 tables accessibles ===
table_name  columns                                     column_list
 suppliers       18 id, name, company_id, company_name, vat_rate...


## Liste colonnes Fournisseurs Pennylane ##

In [6]:
# Afficher les colonnes suppliers
tables_found = [
    'suppliers'
]

for table in tables_found:
    query = f"SELECT * FROM pennylane.{table} LIMIT 0;"  # LIMIT 0 = structure seulement
    df = pd.read_sql(query, conn)
    print(f"\n{'='*60}")
    print(f"Table: pennylane.{table}")
    print(f"{'='*60}")
    print(f"Colonnes ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")


Table: pennylane.suppliers
Colonnes (18):
   1. id
   2. name
   3. company_id
   4. company_name
   5. vat_rate
   6. plan_item_number
   7. pnl_plan_item_number
   8. postal_code
   9. country_alpha2
  10. city
  11. registration_number
  12. vat_number
  13. activity_code
  14. legal_name
  15. registration_country_alpha2
  16. iban_updated_at
  17. iban_updated_by
  18. updated_at


C:\Users\ycloa\AppData\Local\Temp\ipykernel_55928\2955648591.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Chargement des données fournisseurs ##

In [7]:
# Charger les fournisseurs
query_gl = "SELECT * FROM pennylane.suppliers;"
df_suppliers = pd.read_sql(query_gl, get_active_connection())

print(f"✓ Données chargées : {len(df_suppliers)} lignes, {len(df_suppliers.columns)} colonnes")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_55928\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_55928\1910268051.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_suppliers = pd.read_sql(query_gl, get_active_connection())


✓ Données chargées : 50 lignes, 18 colonnes


## Affichage des colonnes disponibles ##

In [8]:
print("Colonnes disponibles :")
for i, col in enumerate(df_suppliers, 1):
    print(f"  {i:2d}. {col}")

Colonnes disponibles :
   1. id
   2. name
   3. company_id
   4. company_name
   5. vat_rate
   6. plan_item_number
   7. pnl_plan_item_number
   8. postal_code
   9. country_alpha2
  10. city
  11. registration_number
  12. vat_number
  13. activity_code
  14. legal_name
  15. registration_country_alpha2
  16. iban_updated_at
  17. iban_updated_by
  18. updated_at


## Affichage type clonne ##

In [9]:
print(f"Types de données des {len(df_suppliers.columns)} colonnes :\n")
for i, (col, dtype) in enumerate(df_suppliers.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

Types de données des 18 colonnes :

   1. id                             → int64
   2. name                           → object
   3. company_id                     → int64
   4. company_name                   → object
   5. vat_rate                       → object
   6. plan_item_number               → object
   7. pnl_plan_item_number           → object
   8. postal_code                    → object
   9. country_alpha2                 → object
  10. city                           → object
  11. registration_number            → object
  12. vat_number                     → object
  13. activity_code                  → object
  14. legal_name                     → object
  15. registration_country_alpha2    → object
  16. iban_updated_at                → datetime64[ns]
  17. iban_updated_by                → object
  18. updated_at                     → datetime64[ns]


## Conversion des types de colonnes ##

In [10]:
# Conversion des types
df_suppliers = df_suppliers.astype({
    'updated_at': 'datetime64[ns]'
})

# Convertir toutes les autres colonnes en string
columns_to_string = [col for col in df_suppliers.columns if col not in ['updated_at']]
df_suppliers[columns_to_string] = df_suppliers[columns_to_string].astype(str)

print("✓ Conversion effectuée\n")
print("Nouveaux types :")
for i, (col, dtype) in enumerate(df_suppliers.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

✓ Conversion effectuée

Nouveaux types :
   1. id                             → object
   2. name                           → object
   3. company_id                     → object
   4. company_name                   → object
   5. vat_rate                       → object
   6. plan_item_number               → object
   7. pnl_plan_item_number           → object
   8. postal_code                    → object
   9. country_alpha2                 → object
  10. city                           → object
  11. registration_number            → object
  12. vat_number                     → object
  13. activity_code                  → object
  14. legal_name                     → object
  15. registration_country_alpha2    → object
  16. iban_updated_at                → object
  17. iban_updated_by                → object
  18. updated_at                     → datetime64[ns]


## Connexion PostgreSQL local ##

In [11]:
# Connexion à PostgreSQL local (utilise les variables du .env)
conn_pg = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    port=int(os.getenv('POSTGRES_PORT')),
    dbname=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD')
)

print("✓ Connexion PostgreSQL établie")
print(f"  Database: {os.getenv('POSTGRES_DB')}")
print(f"  User: {os.getenv('POSTGRES_USER')}")
print(f"  Port: {os.getenv('POSTGRES_PORT')}")

✓ Connexion PostgreSQL établie
  Database: pennylane_data
  User: pennylane_user
  Port: 5433


## Import des données dans PostgreSQL avec SQLAlchemy ##

In [12]:
from sqlalchemy import create_engine

# Créer un engine SQLAlchemy
engine = create_engine(
    f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
)

print("✓ Engine SQLAlchemy créé")

✓ Engine SQLAlchemy créé


## Export du DataFrame vers PostgreSQL ##

## Export vers PostgreSQL local ##

In [ ]:
# Export DataFrame vers PostgreSQL local
df_suppliers.to_sql(
    name='suppliers',
    con=engine,
    schema='pennylane',
    if_exists='replace',
    index=False,
    method='multi',
    chunksize=1000
)

print(f"[OK] Table 'suppliers' exportee : {len(df_suppliers)} lignes")

# Fermer connexions
conn.close()
conn_pg.close()

print("[OK] Connexions fermees")